In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import contractions
import re
import string
import json
from collections import Counter

In [ ]:
model_name = 'cardiffnlp/twitter-roberta-base-emotion-multilabel-latest'
model_file = "twitter-roberta-base-emotion-multilabel-latest"

tokenizer = AutoTokenizer.from_pretrained(model_name)

max_len = 128

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.bert_model = AutoModel.from_pretrained(model_name)
        self.dropout = torch.nn.Dropout(0.4)
        self.linear = torch.nn.Linear(768, 3) # input:embedding length (768), output:number of labels (3)
    def forward(self, input_ids, attention_mask, token_type_ids) :
        bert_output = self.bert_model(input_ids, attention_mask, token_type_ids).pooler_output
        bert_output = self.dropout(bert_output)
        final_output = self.linear(bert_output)
        return final_output

In [ ]:
trained_model = Model()
trained_model.load_state_dict(torch.load(f'./{model_file}.pth', map_location=device))
trained_model.eval()

In [ ]:
# function for getting sentiment category of a text
def get_category(model, text) :
    punctuation_marks = string.punctuation.replace("'","").replace("-","")
    def remove_punctuation(text):
        return text.translate(str.maketrans(punctuation_marks, " "*len(punctuation_marks)))

    def convert_lower(text) :
        return text.lower()

    with open("./chat_words.txt", "r") as f :
        chat_words = json.load(f)

    def expand_chat_words(text) :
        return " ".join([chat_words[i] if i in chat_words else i for i in text.split()])

    def expand_contractions(text):
        return contractions.fix(text)

    def substitute_laugh(text) :
        return re.sub("ha", "laugh ", text)

    def remove_non_latin_emoji(text):
        return re.sub(r"[^A-Za-z- '\U0001F300-\U0001F64F\U0001F680-\U0001F6FF\U0001F910-\U0001F96B\U0001F980-\U0001F9E0]", '', text)

    def preprocessing_pipeline(text) :
        text = text.replace("[NAME]", "").replace("[RELIGION]", "")
        text = remove_punctuation(text)
        text = re.sub(r'[’‘´`“”"]', "'", text)
        text = convert_lower(text)
        text = substitute_laugh(text)
        text = expand_chat_words(text)
        text = expand_contractions(text)
        text = remove_non_latin_emoji(text)
        return text

    t = tokenizer.encode_plus(preprocessing_pipeline(text),
                                add_special_tokens=True,
                                max_length=max_len,
                                padding="max_length",
                                return_token_type_ids=True,
                                return_attention_mask=True,
                                return_tensors="pt",
                                truncation=True)

    input_ids = t["input_ids"].to(device, dtype = torch.long)
    attention_mask = t["attention_mask"].to(device, dtype = torch.long)
    token_type_ids = t["token_type_ids"].to(device, dtype = torch.long)

    calculated_labels = model(input_ids, attention_mask, token_type_ids)
    results = (dict(zip(["positive", "negative", "neutral"],torch.sigmoid(calculated_labels).cpu().detach().numpy().tolist()[0])))

    return max(results, key=results.get)

In [ ]:
def create_sentimental_profile(model, sentences_personA, sentences_personB) :
    # get sentiments for sentences for every person
    categories_personA = [get_category(model, item) for item in sentences_personA]
    categories_personB = [get_category(model, item) for item in sentences_personB]

    # find frequencies of sentiments for every person
    frequencies_personA = Counter(categories_personA)
    frequencies_personB = Counter(categories_personB)

    # find non-negative intensity for every person
    intensity_personA = (frequencies_personA["positive"] + frequencies_personA["neutral"])/len(categories_personA)
    intensity_personB = (frequencies_personB["positive"] + frequencies_personB["neutral"])/len(categories_personB)

    # find interaction intensity as arithmetic mean
    return (intensity_personA + intensity_personB)/2

In [ ]:
dialogue = [
    "Hey, what's up? 😊",
    "Not much. I'm just really frustrated about the programming project we turned in yesterday. 😔",
    "Why is that? What happened? 😟",
    "I feel like you didn't do your part and it really hurt our grade. You were supposed to work on the front-end and you barely did anything. 😞",
    "What are you talking about? I worked really hard on the front-end! You were the one who was supposed to do the back-end and you didn't even finish it!",
    "That's not true! I finished everything on time and it was all working perfectly. You were the one who was slacking off and not contributing anything. 😠🤬",
    "I can't believe you're saying that! I worked just as hard as you did and I did everything I was supposed to do. You're just trying to blame me for your own mistakes. 😠👎🏽",
    "No, I'm not! You're the one who messed everything up and now we're both going to suffer because of it. I can't believe you're being so selfish and stubborn about this. 😠",
    "I'm not being selfish or stubborn! You're just trying to make me look bad so you can feel better about yourself. It's not going to work. We both know what really happened. 🤮",
    "I can't even talk to you right now. You're being so unreasonable and unfair. I thought we were friends, but I guess I was wrong. 😔",
    "I thought so too, but I guess I was wrong too. Maybe we should just work on our own projects from now on. It's obviously not working out between us."
]

# get the intensity of chat
create_sentimental_profile(trained_model, dialogue[::2], dialogue[1::2])